In [ ]:
# Plot ua
# Use kernel geomet-ua 
# Smith Dec 2025. Michael.Smith2@nrcan-rncan.gc.ca
# Retrieves and plots observed/fx soundings from UW and ECCC
# We don't use the ECCC observed UA because of missing data issues
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.plots import SkewT, add_timestamp, Hodograph
from metpy.units import units
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from datetime import datetime, timedelta 


# To-do:
# Add everything to make the prog tephis from ECCC work
# Add a legend to the skewt
# Create a zoomed-in lower atmos output
# Overplot fx soundings on most recent observed.
# Add a lookup file to translate from UW station codes to ECCC. Allow user to only enter the airport code.
# Refactor code to make functions more general
# Figure out why the parcel sounding isn't plotting correctly
# Add calculations to the mix and figure out where to plot them. See https://projectpythia.org/metpy-cookbook/notebooks/skewt/sounding-calculations/
# For some other layout and examples see https://unidata.github.io/MetPy/latest/examples/Advanced_Sounding_With_Complex_Layout.html#sphx-glr-examples-advanced-sounding-with-complex-layout-py 


In [ ]:
# User options
# List of stations for fx: https://dd.weather.gc.ca/20251223/WXO-DD/vertical_profile/doc/station_list_for_vertical_profile.txt
# List of stations for obs can be found at: https://weather.uwyo.edu/upperair/sounding_legacy.html 
stn_id = 'cyxy' # See above. Must be the WMO identifier for UW data, and the ICAO airport code for fx data from ECCC.
date = '20251224'  # YYYYMMDD in UTC
hour = '00'  # HH in UTC. For Canadian sites only 00 and 12 are generally available, while some US sites may have 06 and 18 during the summer convective season.
skew_type = 'fx' # 'obs' or 'fx'


# Check on inputs and exit if invalid
if skew_type.lower() not in ['obs', 'fx']:
    raise ValueError("skew_type must be either 'obs' or 'fx'")

if hour not in ['00','06', '12', '18']:
    raise ValueError("hour must be either '00', '06', '12' or '18'")

date_test = pd.to_datetime(date, format='%Y%m%d', utc=True)  # will raise error if invalid
if not (date_test <= pd.Timestamp.utcnow().normalize()):
    raise ValueError("date must be in the past or present (UTC)")
elif skew_type.lower() == 'fx' and (date_test + pd.Timedelta(days=30)) < (pd.Timestamp.utcnow().normalize() ):
    raise ValueError("Forecast soundings are only available from Datamart for the past 30 days")


In [ ]:
# Build a URL to grab the csv output from UW 
def get_uw_ua(date, hour, stn_id):

    # URL format for UW csv observed UA is
    # https://weather.uwyo.edu/wsgi/sounding?datetime=2025-12-23%2012:00:00&id=71964&type=TEXT:CSV
    date_formatted = f'{date[0:4]}-{date[4:6]}-{date[6:8]}'
    url = f'https://weather.uwyo.edu/wsgi/sounding?datetime={date_formatted}%20{hour}:00:00&id={stn_id}&type=TEXT:CSV'
    return url


# Build a URL to get data from ECCC
def get_eccc_ua(date, hour, stn_id):

    # URL format for ECCC csv forecast UA is
    # https://dd.weather.gc.ca/20251223/WXO-DD/vertical_profile/forecast/csv/ProgTephi_12_71964.csv
    url = f'https://dd.weather.gc.ca/{date}/WXO-DD/vertical_profile/forecast/csv/ProgTephi_{hour}_{stn_id.upper()}.csv'
    return url


# ------ reshape_uw
# Reshape the UW data into a usable df
def reshape_uw_df(df_raw):
    # Convert non-numeric data to NaN in key columns
    key_cols = ['pressure_hPa', 'temperature_C', 'dew point temperature_C', 'wind speed_m/s', 'wind direction_degree']
    for col in key_cols:
        if col in df_raw.columns:
            df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

    df_raw['wind speed_kmh'] = df_raw['wind speed_m/s'].values.astype(float) * 3.6 * units.km / units.h

    return df_raw
    



# ----- Function to reshape the ECCC data into something usable
def reshape_eccc_df(df_raw, fx_hours=[6, 12, 18, 24, 36, 48]):

    df_raw = df_raw.rename(columns={'Variable': 'variable', 'Level': 'pressure', 'Fcst_Hr': 'forecast_hour', 'Value': 'value'})
    df_raw = df_raw.sort_values(['forecast_hour', 'pressure'], ascending=[True, False])
    df_raw = df_raw[df_raw['pressure'].between(100, 1015)]

    dfs = {}
    for fh in sorted(df_raw['forecast_hour'].unique()):
        df_fh = df_raw[df_raw['forecast_hour'] == fh]
        df_pivot = df_fh.pivot(index='pressure', columns='variable', values='value').reset_index()
        df_pivot = df_pivot.sort_values('pressure')

        # Rename columns
        rename_dict = {
            'pressure': 'pressure_hPa',
            'TT': 'temperature_C',
            'WD': 'wind direction_degree',
            'UV': 'wind speed_kt',
            'ES': 'dewpoint_depression_C',
            'GZ': 'geopotential height_dm'
        }
        df_pivot = df_pivot.rename(columns=rename_dict)

        # Calculate dewpoint
        if 'temperature_C' in df_pivot.columns and 'dewpoint_depression_C' in df_pivot.columns:
            df_pivot['dew point temperature_C'] = df_pivot['temperature_C'] - df_pivot['dewpoint_depression_C']
        
        dfs[fh] = df_pivot

    # Combine all forecast hours into one df
    all_dfs = []
    for fh, df_pivot in dfs.items():
        df_pivot['forecast_hour'] = fh
        all_dfs.append(df_pivot)

    df_all = pd.concat(all_dfs, ignore_index=True)

    # Sort: first ascending forecast_hour, then descending pressure
    df_all = df_all.sort_values(['forecast_hour', 'pressure_hPa'], ascending=[True, False])

    # Convert speed from kt to km/h
    df_all['wind speed_kmh'] = df_all['wind speed_kt'].astype(float) * 1.852

    df_all = df_all[df_all['forecast_hour'].isin(fx_hours)]

    return df_all
# --------------------------------------------------

# ------Function to create a name for the output figure
def make_title(type, site, date, hour):
    return f'{site}_{type}_{date}__{hour}UTC_skewT.png'

#------------------------- Plot skew. 
# Some code is from the MetPy Cookbook: https://unidata.github.io/MetPy/latest/examples/skewt_soundings/Skew-T_Soundings.html
# df = a data frame containing pressure (ascending) and corresponding T, Td, WS, WD fields for a single sounding, or for multiple soundings if plot_type = 'all'
# skew_type: <'obs'|'fx'>. String.
# plot_type: <'single'|'all'>. String.  If 'single', the df must contain only one sounding whether it's obs or fx. If 'all', 
#             the script will overplot all soundings in the df and will not overplot sounding calculations (LCL, parcel profile)
# zoom: <True|False>. Boolean. if True, will create a zoomed-in version of the skewt focusing on the lower atmosphere. 
def plot_skewt(df, skew_type='obs', plot_type='single', zoom=False):

    pres = df['pressure_hPa'].values * units.hPa
    temp = df['temperature_C'].values * units.degC
    dewpoint = df['dew point temperature_C'].values * units.degC
    
    wind_speed = df['wind speed_kmh'].values * units.km / units.h
    wind_dir = df['wind direction_degree'].values * units.degrees
    u, v = mpcalc.wind_components(wind_speed, wind_dir)

    # Define the figure and rotation
    fig = plt.Figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=45)

    # plot t, td, wind
    skew.plot(pres, temp, 'red')
    skew.plot(pres, dewpoint, 'blue')

    barb_interval = np.arange(100, 1000, 50) * units('hPa')
    ix = mpcalc.resample_nn_1d(pres, barb_interval)
    skew.plot_barbs(pres[ix], u[ix], v[ix], xloc=1)

    # Calculate and plot LCL and parcel profile
    lcl_pressure, lcl_temperature = mpcalc.lcl(pres[0], temp[0], dewpoint[0])
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

    profile = mpcalc.parcel_profile(pres, temp[1], dewpoint[1]).to('degC')
    skew.plot(pres[1:], profile[1:], 'k', linestyle='dashed', linewidth=2)

    # Tweak the labels and axes
    skew.ax.set_xlabel('Temperature (°C)')
    skew.ax.set_ylabel('Pressure (hPa)')
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-55, 30)

    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

    # Plot adiabats and mixing lines
    skew.plot_dry_adiabats(t0=np.arange(233, 533, 15) * units.K, linewidth=0.5, alpha=0.05, color='orangered')
    skew.plot_moist_adiabats(t0=np.arange(233, 400, 10) * units.K, linewidth=0.5, alpha=0.05, color='tab:green')
    skew.plot_mixing_lines(pressure=np.arange(1000, 99, -25) * units.hPa, linewidth=0.5, linestyle='dotted', color='tab:blue')

    # Add geopotential height labels on the primary y-axis
    target_pressures = [1000, 850, 700, 500, 300, 200, 100]
    pres_df = df['pressure_hPa'].dropna().values
    height_df = df['geopotential height_m'].dropna().values
    if len(pres_df) > 1 and len(height_df) > 1:
        for p in target_pressures:
            if p >= pres_df.min() and p <= pres_df.max():
                h = np.interp(p, pres_df[::-1], height_df[::-1])
                h_dm = h / 10  # Convert to decameters
                skew.ax.text(-50, p, f'{h_dm:.0f} dm', fontsize=9, color='gray', ha='left', va='center')

    
    # Add a hodograph to the top left
    ax_hod = inset_axes(skew.ax, '35%','35%', loc=1)
    h = Hodograph(ax_hod, component_range=80)
    h.add_grid(increment=20)
    h.plot_colormapped(u, v, pres, cmap='viridis')
    
    
    return skew

In [ ]:
# Main
if __name__ == '__main__':

    # Call the appropriate functions to retrieve data and shape it prior to plotting and calcs
    if skew_type.lower() == 'obs':
        url = get_uw_ua(date, hour, stn_id)
        df_raw = pd.read_csv(url, sep=',', header=0)
        df = reshape_uw_df(df_raw)
    elif skew_type.lower() == 'fx':
        url = get_eccc_ua(date, hour, stn_id)
        df_raw = pd.read_csv(url, header=1, skiprows=[2])
        df = reshape_eccc_df(df_raw)
        #print(df.head(20))
        #df.to_csv("foo.csv")



    #skewt = plot_skewt(df, skew_type=skew_type)

    #skewt.ax.set_title(f'{skew_type.upper()} Skew-T for Station {stn_id} valid {date} {hour}UTC', fontsize='large')
    #skewt.ax.set_adjustable

    #current_utc = pd.Timestamp.utcnow()
    #add_timestamp(skewt.ax, time=current_utc, y=-0.10, x=0.0, ha='left', time_format='%Y-%m-%d %H:%M UTC', fontsize='medium')

    ## Add label for secondary y-axis (height)
    #skewt.ax.text(1.08, 0.5, 'Wind (km/h)', transform=skewt.ax.transAxes, rotation=90, va='center', ha='left', fontsize='medium')

    #skewt.ax.figure.savefig(make_title(skew_type, stn_id, date, hour))